In [93]:
#-*-coding:utf-8-*-

import numpy as py
import pandas as pd
import matplotlib as mpl

import time
import datetime
import gc

from sklearn.model_selection import  KFold, cross_val_score, train_test_split
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss

print 'start...'

start...


In [94]:
train=pd.read_table('/Users/yellow/Downloads/给选手/round1_iflyad_train.txt')

test=pd.read_table('/Users/yellow/Downloads/给选手/round1_iflyad_test_feature.txt')

# print train.info()

In [95]:
data=pd.concat([train,test],ignore_index=True)
# print train.shape
# print test.shape
# print data.shape

/Users/yellow/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [96]:
#数据处理

#1.处理缺失值
data['make']=data['make'].fillna(str(-1))
data['model'] = data['model'].fillna(str(-1))
data['osv'] = data['osv'].fillna(str(-1))
data['app_cate_id'] = data['app_cate_id'].fillna(-1)
data['app_id'] = data['app_id'].fillna(-1)
data['click'] = data['click'].fillna(-1)
data['user_tags'] = data['user_tags'].fillna(str(-1))
data['f_channel'] = data['f_channel'].fillna(str(-1))

print '...'

...


In [97]:
#替换 bool值，pd里面的bool值为 true false

replace=['creative_is_jump', 'creative_is_download', 'creative_is_js', 'creative_is_voicead', 'creative_has_deeplink', 'app_paid']
for item in replace:
    data[item]=data[item].replace([False,True],[0,1])


# print data.info()

In [98]:
#labelencode 把一些类别特征转化成数字
#LabelEncoder可以将标签分配一个0—n_classes-1之间的编码 ,将各种标签分配一个可数的连续编号
from sklearn import preprocessing

encoder = ['city', 'province', 'make', 'model', 'osv', 'os_name', 'adid', 'advert_id', 'orderid',
           'advert_industry_inner', 'campaign_id', 'creative_id', 'app_cate_id',
           'app_id', 'inner_slot_id', 'advert_name', 'f_channel', 'creative_tp_dnf']

label_encode=preprocessing.LabelEncoder()
for item in encoder:
    label_encode.fit(data[item])
    data[item] = label_encode.transform(data[item])

# print data.head(5)


In [23]:
# print int(time.strftime("%d", time.localtime(2190219034)))
# print time.localtime(2190219034)

29
time.struct_time(tm_year=2039, tm_mon=5, tm_mday=29, tm_hour=2, tm_min=10, tm_sec=34, tm_wday=6, tm_yday=149, tm_isdst=0)


In [99]:
# 构造特征 ，也就是在原有特征的基础上，构造出一些新的特征

data['day']=data['time'].apply(lambda x :int(time.strftime("%d", time.localtime(x))) )
data['hour']=data['time'].apply(lambda x : int(time.strftime("%H", time.localtime(x))))

# print data.info()

In [100]:
#这里是观察数据的出来的，也就是天数跨了月份，所以这么处理是为了保持时间序列是有序的
data['period'] = data['day']
data['period'][data['period']<27] = data['period'][data['period']<27] + 31

#####给出点击率，至于哪些特征需要给出点击率，也是从数据中观察出来的
click_rate=['advert_id','advert_industry_inner','advert_name','campaign_id', 'creative_height','creative_tp_dnf', 'creative_width', 'province', 'f_channel']

for feat_1 in click_rate:
    
    gc.collect()
    res=pd.DataFrame()
    temp=data[[feat_1,'period','click']]
    
    for period in range(27,35):#这里的意思是，构造 feat_1 特征从 27 到 34 每天的点击率
        
        #这里的意思估计是，小于等于 27 号的全部当成 27号那一天
        if period == 27:
            
            #这个特征的数目（因为这个特征不是在全部样本中都出现的）count
            count=temp.groupby([feat_1]).apply(lambda x: x['click'][(x['period']<=period).values].count()).reset_index(name=feat_1+'_all')           
            #这个特征的点击数目  sum
            count1=temp.groupby([feat_1]).apply(lambda x: x['click'][(x['period']<=period).values].sum()).reset_index(name=feat_1+'_1')
        else:
            count=temp.groupby([feat_1]).apply(lambda x: x['click'][(x['period']<period).values].count()).reset_index(name=feat_1+'_all')
            count1=temp.groupby([feat_1]).apply(lambda x: x['click'][(x['period']<period).values].sum()).reset_index(name=feat_1+'_1')
            
            
        count[feat_1+'_1']=count1[feat_1+'_1']
        count.fillna(value=0, inplace=True)
        
       # print type(round(count[feat_1+'_1'] / count[feat_1+'_all'], 5))
        
#         count[feat_1+'_rate'] = round(count[feat_1+'_1'] / count[feat_1+'_all'], 5)
        count[feat_1+'_rate'] = count[feat_1+'_1'] / count[feat_1+'_all']
        count['period']=period
        count.drop([feat_1+'_all', feat_1+'_1'],axis=1,inplace=True)
        count.fillna(value=0, inplace=True)
        #追加，只能在行上合并，是concat的简略版
        res=res.append(count,ignore_index=True)
        
    
    print(feat_1,' over')
    data = pd.merge(data,res, how='left', on=[feat_1,'period'])


/Users/yellow/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


('advert_id', ' over')
('advert_industry_inner', ' over')
('advert_name', ' over')
('campaign_id', ' over')
('creative_height', ' over')
('creative_tp_dnf', ' over')
('creative_width', ' over')
('province', ' over')
('f_channel', ' over')


In [43]:
# temp=data[['advert_id','period','click']]
# test=temp.groupby(['advert_id']).apply(lambda x: x['click'][(x['period']<=27).values].count()).reset_index(name='feat_1'+'_all')
# print test.head()
# print (test['advert_id']<=4).values
# [ True  True  True  True  True False False False False False False False
#  False False False False False False False False False False False False
#  False False False False False False False False False False False False
#  False False False]

[ True  True  True  True  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False]


In [48]:
print data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041674 entries, 0 to 1041673
Data columns (total 47 columns):
adid                          1041674 non-null int64
advert_id                     1041674 non-null int64
advert_industry_inner         1041674 non-null int64
advert_name                   1041674 non-null int64
app_cate_id                   1041674 non-null int64
app_id                        1041674 non-null int64
app_paid                      1041674 non-null int64
campaign_id                   1041674 non-null int64
carrier                       1041674 non-null int64
city                          1041674 non-null int64
click                         1041674 non-null float64
creative_has_deeplink         1041674 non-null int64
creative_height               1041674 non-null int64
creative_id                   1041674 non-null int64
creative_is_download          1041674 non-null int64
creative_is_js                1041674 non-null int64
creative_is_jump              1041674

In [101]:
#删除一些没用的特征

drop = ['click', 'time', 'instance_id', 'user_tags',  'app_paid', 'creative_is_js', 'creative_is_voicead']

#训练集
train=data[:train.shape[0]]
y_train=train.loc[:,"click"]

#测试集
test=data[train.shape[0]:]
res=test.loc[:,['instance_id']]

train.drop(drop,axis=1,inplace=True)
test.drop(drop,axis=1,inplace=True)

print train.shape
print test.shape

print len(res['instance_id'])

(1001650, 40)
(40024, 40)
40024


In [102]:
#模型构建
import lightgbm as lgb

print 'model......'

model......


In [103]:

#这些是model的超参数
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=10, silent=True)
#help(lgb.LGBMClassifier)

#构建用于训练模型的数据，用分层采样来形成 伍分数据，交叉验证

X_loc_train = train.values
y_loc_train = y_train.values
X_loc_test = test.values

#返回的是索引  shuffle=True,random_state=1024 无论执行几次，划分的结构都是一样的，方便调整上面的参数吧
#这个的作用是返回五份数据，每份数据包含两份，一份是80%，一份是20%
#注意：按照理论上说，是一次把数据分成5分，然后组装成不同 80&20
    
# 0
# [      0       1       2 ... 1001646 1001647 1001648]
# [      7      10      11 ... 1001641 1001643 1001649]

skf=list(StratifiedKFold(y_loc_train,n_folds=5,shuffle=True,random_state=1024))

print len(skf)
print len(skf[0])
print type(skf[0])
# 5
# 2
# <type 'tuple'>

# help(model.fit)

5
2
<type 'tuple'>


In [104]:

#训练模型

baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), (X_loc_train[test_index], y_loc_train[test_index])],
                          early_stopping_rounds=100)
    
    #eval_names 和 eval_set是对应的，eval_set是用来做验证的，包括在训练集和测试集上的验证
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    
    #这里加起来，应该是整个测试集和的总误差
    loss += lgb_model.best_score_['valid']['binary_logloss']
    
    #这里预测的是概率，而不是一个具体的类别
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    
    res['prob_%s' % str(i)] = test_pred#这是一个 narray

print 'train ok......'
    



    
    

('Fold', 0)
[1]	train's binary_logloss: 0.491932	train's binary_logloss: 0.491932	valid's binary_logloss: 0.491971	valid's binary_logloss: 0.491971
Training until validation scores don't improve for 100 rounds.
[2]	train's binary_logloss: 0.486234	train's binary_logloss: 0.486234	valid's binary_logloss: 0.486311	valid's binary_logloss: 0.486311
[3]	train's binary_logloss: 0.481122	train's binary_logloss: 0.481122	valid's binary_logloss: 0.481222	valid's binary_logloss: 0.481222
[4]	train's binary_logloss: 0.476472	train's binary_logloss: 0.476472	valid's binary_logloss: 0.476606	valid's binary_logloss: 0.476606
[5]	train's binary_logloss: 0.472228	train's binary_logloss: 0.472228	valid's binary_logloss: 0.47238	valid's binary_logloss: 0.47238
[6]	train's binary_logloss: 0.468371	train's binary_logloss: 0.468371	valid's binary_logloss: 0.468544	valid's binary_logloss: 0.468544
[7]	train's binary_logloss: 0.464822	train's binary_logloss: 0.464822	valid's binary_logloss: 0.465019	valid's 

[61]	train's binary_logloss: 0.419317	train's binary_logloss: 0.419317	valid's binary_logloss: 0.420067	valid's binary_logloss: 0.420067
[62]	train's binary_logloss: 0.419214	train's binary_logloss: 0.419214	valid's binary_logloss: 0.419971	valid's binary_logloss: 0.419971
[63]	train's binary_logloss: 0.419101	train's binary_logloss: 0.419101	valid's binary_logloss: 0.419872	valid's binary_logloss: 0.419872
[64]	train's binary_logloss: 0.419009	train's binary_logloss: 0.419009	valid's binary_logloss: 0.419796	valid's binary_logloss: 0.419796
[65]	train's binary_logloss: 0.418912	train's binary_logloss: 0.418912	valid's binary_logloss: 0.419717	valid's binary_logloss: 0.419717
[66]	train's binary_logloss: 0.418819	train's binary_logloss: 0.418819	valid's binary_logloss: 0.419639	valid's binary_logloss: 0.419639
[67]	train's binary_logloss: 0.418744	train's binary_logloss: 0.418744	valid's binary_logloss: 0.419581	valid's binary_logloss: 0.419581
[68]	train's binary_logloss: 0.41866	trai

[121]	train's binary_logloss: 0.416266	train's binary_logloss: 0.416266	valid's binary_logloss: 0.417852	valid's binary_logloss: 0.417852
[122]	train's binary_logloss: 0.416237	train's binary_logloss: 0.416237	valid's binary_logloss: 0.417844	valid's binary_logloss: 0.417844
[123]	train's binary_logloss: 0.416205	train's binary_logloss: 0.416205	valid's binary_logloss: 0.417824	valid's binary_logloss: 0.417824
[124]	train's binary_logloss: 0.416174	train's binary_logloss: 0.416174	valid's binary_logloss: 0.417808	valid's binary_logloss: 0.417808
[125]	train's binary_logloss: 0.416149	train's binary_logloss: 0.416149	valid's binary_logloss: 0.417795	valid's binary_logloss: 0.417795
[126]	train's binary_logloss: 0.416125	train's binary_logloss: 0.416125	valid's binary_logloss: 0.417787	valid's binary_logloss: 0.417787
[127]	train's binary_logloss: 0.4161	train's binary_logloss: 0.4161	valid's binary_logloss: 0.41777	valid's binary_logloss: 0.41777
[128]	train's binary_logloss: 0.416079	t

[181]	train's binary_logloss: 0.414837	train's binary_logloss: 0.414837	valid's binary_logloss: 0.417373	valid's binary_logloss: 0.417373
[182]	train's binary_logloss: 0.414812	train's binary_logloss: 0.414812	valid's binary_logloss: 0.417362	valid's binary_logloss: 0.417362
[183]	train's binary_logloss: 0.414794	train's binary_logloss: 0.414794	valid's binary_logloss: 0.417363	valid's binary_logloss: 0.417363
[184]	train's binary_logloss: 0.414775	train's binary_logloss: 0.414775	valid's binary_logloss: 0.417366	valid's binary_logloss: 0.417366
[185]	train's binary_logloss: 0.414752	train's binary_logloss: 0.414752	valid's binary_logloss: 0.417364	valid's binary_logloss: 0.417364
[186]	train's binary_logloss: 0.414736	train's binary_logloss: 0.414736	valid's binary_logloss: 0.417364	valid's binary_logloss: 0.417364
[187]	train's binary_logloss: 0.414715	train's binary_logloss: 0.414715	valid's binary_logloss: 0.417358	valid's binary_logloss: 0.417358
[188]	train's binary_logloss: 0.41

[241]	train's binary_logloss: 0.413678	train's binary_logloss: 0.413678	valid's binary_logloss: 0.417102	valid's binary_logloss: 0.417102
[242]	train's binary_logloss: 0.413661	train's binary_logloss: 0.413661	valid's binary_logloss: 0.417107	valid's binary_logloss: 0.417107
[243]	train's binary_logloss: 0.413644	train's binary_logloss: 0.413644	valid's binary_logloss: 0.417103	valid's binary_logloss: 0.417103
[244]	train's binary_logloss: 0.413628	train's binary_logloss: 0.413628	valid's binary_logloss: 0.417103	valid's binary_logloss: 0.417103
[245]	train's binary_logloss: 0.413608	train's binary_logloss: 0.413608	valid's binary_logloss: 0.417101	valid's binary_logloss: 0.417101
[246]	train's binary_logloss: 0.413591	train's binary_logloss: 0.413591	valid's binary_logloss: 0.417096	valid's binary_logloss: 0.417096
[247]	train's binary_logloss: 0.413576	train's binary_logloss: 0.413576	valid's binary_logloss: 0.417092	valid's binary_logloss: 0.417092
[248]	train's binary_logloss: 0.41

...

[440]	train's binary_logloss: 0.410491	train's binary_logloss: 0.410491	valid's binary_logloss: 0.416958	valid's binary_logloss: 0.416958
[441]	train's binary_logloss: 0.410476	train's binary_logloss: 0.410476	valid's binary_logloss: 0.416957	valid's binary_logloss: 0.416957
[442]	train's binary_logloss: 0.410462	train's binary_logloss: 0.410462	valid's binary_logloss: 0.41695	valid's binary_logloss: 0.41695
[443]	train's binary_logloss: 0.410449	train's binary_logloss: 0.410449	valid's binary_logloss: 0.416951	valid's binary_logloss: 0.416951
[444]	train's binary_logloss: 0.410436	train's binary_logloss: 0.410436	valid's binary_logloss: 0.416951	valid's binary_logloss: 0.416951
[445]	train's binary_logloss: 0.410421	train's binary_logloss: 0.410421	valid's binary_logloss: 0.416951	valid's binary_logloss: 0.416951
[446]	train's binary_logloss: 0.410408	train's binary_logloss: 0.410408	valid's binary_logloss: 0.416953	valid's binary_logloss: 0.416953
[447]	train's binary_logloss: 0.4103

[500]	train's binary_logloss: 0.409674	train's binary_logloss: 0.409674	valid's binary_logloss: 0.416942	valid's binary_logloss: 0.416942
[501]	train's binary_logloss: 0.409661	train's binary_logloss: 0.409661	valid's binary_logloss: 0.416946	valid's binary_logloss: 0.416946
[502]	train's binary_logloss: 0.409648	train's binary_logloss: 0.409648	valid's binary_logloss: 0.416946	valid's binary_logloss: 0.416946
[503]	train's binary_logloss: 0.409635	train's binary_logloss: 0.409635	valid's binary_logloss: 0.416947	valid's binary_logloss: 0.416947
[504]	train's binary_logloss: 0.409623	train's binary_logloss: 0.409623	valid's binary_logloss: 0.416948	valid's binary_logloss: 0.416948
[505]	train's binary_logloss: 0.409608	train's binary_logloss: 0.409608	valid's binary_logloss: 0.416952	valid's binary_logloss: 0.416952
[506]	train's binary_logloss: 0.409597	train's binary_logloss: 0.409597	valid's binary_logloss: 0.416949	valid's binary_logloss: 0.416949
[507]	train's binary_logloss: 0.40

[560]	train's binary_logloss: 0.408864	train's binary_logloss: 0.408864	valid's binary_logloss: 0.416926	valid's binary_logloss: 0.416926
[561]	train's binary_logloss: 0.408852	train's binary_logloss: 0.408852	valid's binary_logloss: 0.416928	valid's binary_logloss: 0.416928
[562]	train's binary_logloss: 0.408839	train's binary_logloss: 0.408839	valid's binary_logloss: 0.416927	valid's binary_logloss: 0.416927
[563]	train's binary_logloss: 0.408825	train's binary_logloss: 0.408825	valid's binary_logloss: 0.416928	valid's binary_logloss: 0.416928
[564]	train's binary_logloss: 0.408815	train's binary_logloss: 0.408815	valid's binary_logloss: 0.416924	valid's binary_logloss: 0.416924
[565]	train's binary_logloss: 0.408801	train's binary_logloss: 0.408801	valid's binary_logloss: 0.416922	valid's binary_logloss: 0.416922
[566]	train's binary_logloss: 0.408788	train's binary_logloss: 0.408788	valid's binary_logloss: 0.416924	valid's binary_logloss: 0.416924
[567]	train's binary_logloss: 0.40

[620]	train's binary_logloss: 0.408093	train's binary_logloss: 0.408093	valid's binary_logloss: 0.416932	valid's binary_logloss: 0.416932
[621]	train's binary_logloss: 0.408082	train's binary_logloss: 0.408082	valid's binary_logloss: 0.416932	valid's binary_logloss: 0.416932
[622]	train's binary_logloss: 0.408069	train's binary_logloss: 0.408069	valid's binary_logloss: 0.416933	valid's binary_logloss: 0.416933
[623]	train's binary_logloss: 0.408052	train's binary_logloss: 0.408052	valid's binary_logloss: 0.416935	valid's binary_logloss: 0.416935
[624]	train's binary_logloss: 0.408039	train's binary_logloss: 0.408039	valid's binary_logloss: 0.416936	valid's binary_logloss: 0.416936
[625]	train's binary_logloss: 0.408027	train's binary_logloss: 0.408027	valid's binary_logloss: 0.416935	valid's binary_logloss: 0.416935
[626]	train's binary_logloss: 0.408011	train's binary_logloss: 0.408011	valid's binary_logloss: 0.416933	valid's binary_logloss: 0.416933
[627]	train's binary_logloss: 0.40

[680]	train's binary_logloss: 0.40733	train's binary_logloss: 0.40733	valid's binary_logloss: 0.416934	valid's binary_logloss: 0.416934
Early stopping, best iteration is:
[580]	train's binary_logloss: 0.408607	train's binary_logloss: 0.408607	valid's binary_logloss: 0.416912	valid's binary_logloss: 0.416912
train ok......


In [105]:
#上面再 train 数据上训练和测试 ，并且获得了在 test 数据上的预测数据，这就是我们要提交的结果
#这里把五个模型的结果取平均值

res['total_prop']=0

print type(res['prob_1'])
for item in range(0,5):
    res['total_prop']+=res['prob_%s' % str(i)]

res['real_prop']=res['total_prop']/5

print res['real_prop']

<class 'pandas.core.series.Series'>
1001650    0.111946
1001651    0.291843
1001652    0.115001
1001653    0.221907
1001654    0.003478
1001655    0.129077
1001656    0.006760
1001657    0.261818
1001658    0.348202
1001659    0.338908
1001660    0.286637
1001661    0.166600
1001662    0.432805
1001663    0.005668
1001664    0.196625
1001665    0.003512
1001666    0.211472
1001667    0.335188
1001668    0.493514
1001669    0.295676
1001670    0.162505
1001671    0.281431
1001672    0.040213
1001673    0.237650
1001674    0.247217
1001675    0.095439
1001676    0.527300
1001677    0.003224
1001678    0.168512
1001679    0.252774
             ...   
1041644    0.292798
1041645    0.056613
1041646    0.007154
1041647    0.177691
1041648    0.006865
1041649    0.537920
1041650    0.220986
1041651    0.283122
1041652    0.301648
1041653    0.185900
1041654    0.427205
1041655    0.272416
1041656    0.028422
1041657    0.251878
1041658    0.493638
1041659    0.368183
1041660    0.228195
1041

In [106]:
#提交上面获取到的结果
#res=test.loc[:,'instance_id']

now=datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
print now

print len(res['real_prop'])
print len(res['instance_id'])
#res[['instance_id', 'predicted_score']] 这样获取到的是一个dateframe
res[['instance_id', 'real_prop']].to_csv('/Users/yellow/Downloads/给选手/base_line5.csv',index=False)

09-21-14-58
40024
40024
